In [15]:
import pandas as pd
from openai import OpenAI
import os

from typing import Literal

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [6]:
class FridgeUpdateInput(BaseModel):
    item: str = Field(description="Item that is updated in the fridge. Make sure it is NOT plural, but a singular word")
    count_type: Literal["add", "remove", "total"] = Field(description="Add, remove or set the total of the number of items given.")
    quantity: int = Field(description="The number of items in the user query. E.g. if -- remove 3 milks -- is the query, set it to 3. In a special case, if -- remove all -- or something similar is stated, set to 10000")


In [7]:
@tool("fridge-update-tool", args_schema=FridgeUpdateInput, return_direct=True)
def update_fridge(item: str, count_type: str, quantity: int) -> str:
    """Update the contents of the fridge by adding to, removing from, or setting the number of items in the inventory"""

    print(f"FUNCTION CALLED: {item}, {count_type}, {quantity}")
    df = pd.read_csv('./data/fridge_inventory.csv')
    
    if count_type == "add":
        if item in df['item'].values:
            df.loc[df['item'] == item, 'quantity'] += quantity
        else:
            df = pd.concat([df, pd.DataFrame([{'item': item, 'quantity': quantity}])], ignore_index=True)
    elif count_type == "remove":
        if item not in df['item'].values:
            raise ValueError(f"No such item '{item}' exists in the inventory.")
        else:
            current_quantity = df.loc[df['item'] == item, 'quantity'].values[0]
            new_quantity = current_quantity - quantity
            if new_quantity < 0:
                raise ValueError(f"Cannot remove {quantity} {item}(s). Inventory has only {current_quantity} {item}(s).")
            df.loc[df['item'] == item, 'quantity'] = new_quantity
    elif count_type == "total":
        if item not in df['item'].values:
            df = pd.concat([df, pd.DataFrame([{'item': item, 'quantity': quantity}])], ignore_index=True)
        else:
            df.loc[df['item'] == item, 'quantity'] = quantity
    else:
        raise ValueError("Invalid count_type. Should be 'add', 'remove', or 'total'.")

    
    df.to_csv('./data/fridge_inventory.csv',index = False)
    return "Fridge Updated"

In [13]:
tools = [update_fridge]

In [21]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")

#os.environ["OPENAI_API_KEY"] # ADD API KEY


client = OpenAI()

agent = create_openai_tools_agent(client, tool, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


AttributeError: 'OpenAI' object has no attribute 'bind'